In [ ]:
import pandas as pd
from pathlib import Path

gap_df = pd.read_csv(Path('times_result_0.csv'))
grid_size_df = pd.read_csv(Path('times_result_1.csv'))
energy_df = pd.read_csv(Path('times_result_2.csv'))
voltage_df = pd.read_csv(Path('times_result_3.csv'))

: 